In [ ]:
from kl_decomposition import kernel_fit, orthopoly
import numpy as np
import matplotlib.pyplot as plt

# Exponential kernel on $(0,1)$

In [ ]:
def target_function(x): return np.exp(-x)
x, w = orthopoly.gauss_legendre_rule_multilevel(0.0, 1.0, 100, L=5, ratio=0.2)
target_exp = target_function(x)
obj, grad, hess = kernel_fit.build_se_obj_grad_hess(x, w, target_exp)
exp_all_a, exp_all_b, exp_all_vals = kernel_fit.square_exp_approximations_newton(
    max_terms=20, precision=1e-6,
    x=x, w=w, target=target_exp,
    obj=obj, grad=grad, hess=hess, verbose=True)

# Matérn covariance function with $\nu=5/2$ and $\sigma^2=1$ on $(0,1)$


In [ ]:
def target_function(x):
    sqrt5_r = np.sqrt(5) * x
    return (1 + sqrt5_r + (5 / 3) * x**2) * np.exp(-sqrt5_r)


x, w = orthopoly.gauss_legendre_rule_multilevel(0.0, 1.0, 100, L=5, ratio=0.2)
target_matern = target_function(x)
obj, grad, hess = kernel_fit.build_se_obj_grad_hess(x, w, target_matern)
matern_all_a, matern_all_b, matern_all_vals = kernel_fit.square_exp_approximations_newton(
    max_terms=20, precision=1e-6,
    x=x, w=w, target=target_matern,
    obj=obj, grad=grad, hess=hess, verbose=True)

# Powered-exponential covariance function with γ=0.6 and σ²=1 on $(0,1)$

In [ ]:
def target_function(x):
    return np.exp(- x ** 0.6)


x, w = orthopoly.gauss_legendre_rule_multilevel(0.0, 1.0, 100, L=5, ratio=0.2)
target_powexp06 = target_function(x)

obj, grad, hess = kernel_fit.build_se_obj_grad_hess(x, w, target_powexp06)

powexp06_all_a, powexp06_all_b, powexp06_all_vals = kernel_fit.square_exp_approximations_newton(
    max_terms=20, precision=1e-6,
    x=x, w=w, target=target_powexp06,
    obj=obj, grad=grad, hess=hess, verbose=True)

# Rational-quadratic covariance function with α=1 and σ²=1 on $(0,1)$

In [ ]:
def target_function(x):
    return (1 + (x**2) / 2) ** -1


x, w = orthopoly.gauss_legendre_rule_multilevel(0.0, 1.0, 100, L=5, ratio=0.2)
target_rq = target_function(x)

obj, grad, hess = kernel_fit.build_se_obj_grad_hess(x, w, target_rq)

rq_all_a, rq_all_b, rq_all_vals = kernel_fit.square_exp_approximations_newton(
    max_terms=20, precision=1e-6,
    x=x, w=w, target=target_rq,
    obj=obj, grad=grad, hess=hess, verbose=True)

# Generalised Cauchy covariance function with γ=1, β=1 and σ²=1 on $(0,1)$

In [ ]:
def target_function(x):
    return (1 + x) ** -1


x, w = orthopoly.gauss_legendre_rule_multilevel(0.0, 1.0, 100, L=5, ratio=0.2)
target_cauchy = target_function(x)

obj, grad, hess = kernel_fit.build_se_obj_grad_hess(x, w, target_cauchy)

cauchy_all_a, cauchy_all_b, cauchy_all_vals = kernel_fit.square_exp_approximations_newton(
    max_terms=20, precision=1e-6,
    x=x, w=w, target=target_cauchy,
    obj=obj, grad=grad, hess=hess, verbose=True)

# Visualize the covariance functions

In [ ]:
# === PLOT SETTINGS ===========================================================
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.size": 11,
    "axes.labelsize": 11,
    "legend.fontsize": 11,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
})

# === COLLECT DATA ============================================================
# Ensure x is sorted so curves are monotone in r
r_sorted_idx = np.argsort(x)
r = x[r_sorted_idx]

cov_dict = {
    r"$e^{-r}$": target_exp[r_sorted_idx],
    r"Matérn $\nu=\frac{5}{2}$": target_matern[r_sorted_idx],
    r"$\exp bigl(-r^{0.6} bigr)$": target_powexp06[r_sorted_idx],
    r"RQ $(\alpha=1)$": target_rq[r_sorted_idx],
    r"Cauchy $(\gamma=\beta=1)$": target_cauchy[r_sorted_idx],
}

conv_dict = {
    r"$e^{-r}$": exp_all_vals,
    r"Matérn $\nu=\frac{5}{2}$": matern_all_vals,
    r"$\exp bigl(-r^{0.6} bigr)$": powexp06_all_vals,
    r"RQ $(\alpha=1)$": rq_all_vals,
    r"Cauchy $(\gamma=\beta=1)$": cauchy_all_vals,
}

# === FIGURE 1: COVARIANCE CURVES ============================================
fig1, ax1 = plt.subplots(figsize=(3.4, 2.8), dpi=300)   # ~85 mm column width
for label, C in cov_dict.items():
    ax1.plot(r, C, lw=1.2, label=label)

ax1.set_xlabel(r"$r$")
ax1.set_ylabel(r"$C(r)$")
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1.02)
ax1.grid(alpha=0.3)
ax1.legend(frameon=False, loc="upper right", handlelength=1.3)

plt.show()

In [ ]:

fig1.tight_layout()
fig1.savefig("covariance_curves.pdf", bbox_inches="tight")

# === FIGURE 2: CONVERGENCE CURVES ===========================================
fig2, ax2 = plt.subplots(figsize=(3.4, 2.8), dpi=300)

for label, vals in conv_dict.items():
    it = np.arange(1, len(vals) + 1)
    ax2.semilogy(it, vals / vals[0], marker="o", ms=3, lw=1, label=label)

ax2.set_xlabel(r"Iteration $k$")
ax2.set_ylabel(r"$J_k/J_0$ (relative error)")
ax2.set_xlim(1, max(len(v) for v in conv_dict.values()))
ax2.grid(alpha=0.3, which="both")
ax2.legend(frameon=False, loc="upper right", handlelength=1.3)

fig2.tight_layout()
fig2.savefig("convergence_curves.pdf", bbox_inches="tight")
plt.show()